In [1]:
import os
os.chdir("../src/")
import pandas as pd
import s3fs
import import_inpi as inpi

In [3]:
pm, rep = inpi.import_inpi_s3()

/home/coder/work/data_inpi/src/import_inpi.py:109: DtypeWarning: Columns (7,8,10,11,16,30) have mixed types. Specify dtype option on import or set low_memory=False.
  personnes_morales = pd.read_csv(file_in, sep=";")
/home/coder/work/data_inpi/src/import_inpi.py:111: DtypeWarning: Columns (7,15,17,25,26,32,33,34,35,36,37,38,39,40,41,42,43,45,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  representants = pd.read_csv(file_in, sep=";")


In [4]:
pm.shape

(5001845, 31)

In [5]:
rep.shape

(11142886, 51)

In [41]:

def clean_rep_inpi(df_rep):
    """_summary_Function that cleans data from Inpi

    Args:
        df_rep (_type_): _description_

    Returns:
        _type_: _description_
    """
    df_rep.columns = [c.lower() for c in df_rep.columns]
    df_rep = df_rep.drop(['rep_perm_nom',
       'rep_perm_nom_usage', 'rep_perm_pseudo', 'rep_perm_prénoms',
       'rep_perm_date_naissance', 'rep_perm_ville_naissance',
       'rep_perm_pays_naissance', 'rep_perm_nationalité',
       'rep_perm_adr_ligne1', 'rep_perm_adr_ligne2', 'rep_perm_adr_ligne3',
       'rep_perm_code_postal', 'rep_perm_ville', 'rep_perm_code_commune',
       'rep_perm_pays', 'conjoint_collab_nom_patronym',
       'conjoint_collab_nom_usage', 'conjoint_collab_pseudo',
       'conjoint_collab_prénoms', 'conjoint_collab_date_fin'], axis=1)
    df_rep = df_rep.rename({"code greffe": "code_greffe", 
                                "forme_juridique_x": "forme_juridique", 
                                "prénoms": "prenoms", "qualité": "qualite", 
                                "dénomination": "denomination",
                                "nationalité": "nationalite",
                                "id_représentant": "id_representant",
                                "siren.1": "siren_representant_pm"}, axis=1)
    # Traitement des cas où le nom et les prénoms sont manquants
    df_rep.loc[df_rep["prenoms"] == "-", "prenoms"] = ""
    re1 = "(INDIVISION SUCCESSORALE DE M\.\s|\sM\.\s|^M\.\s|MME\.\s|MR\.\s|MELLE\.\s|MONSIEUR\.?\s|MADAME\.?\s|MLLE\.\s|REPRESENTEE\sPAR\sMONSIEUR\.?\s|REPRESENTEE\sPAR\s|SUCCESSION\s|SUCCESSION\sDE\s)"
    # df_rep["top_civilite"] = df_rep["nom_patronymique"].str.contains(re1, regex=True)
    df_rep["nom_patronymique_cor"] = df_rep["nom_patronymique"]
    df_rep["nom_patronymique_cor"] = df_rep["nom_patronymique_cor"].str.replace(re1,"")
    # on a modifié un peu le code de l'insee pour enlever les representee par, ici aussi car on va essayer de prendre en compte les particules
    particule = "(DU\s[A-Za-z]+|DE\sLA\s[A-Za-z]+|[A-Za-z]+\sDU\s[A-Za-z]+|[A-Za-z]+\sDE\sLA\s[A-Za-z]+|DE\s[A-Za-z]+|[A-Za-z]+\sDE\s[A-Za-z]+)"
    df_rep["top_particule"] = df_rep["nom_patronymique"].str.contains(particule, regex=True)
    # Traitement de la qualite
    df_rep["qualite_cor"] = df_rep["qualite"]
    df_rep["qualite_cor"] = df_rep["qualite_cor"].str.lower()
    df_rep["qualite_cor"] = df_rep["qualite_cor"].str.strip()
    # nettoyage de la dénomination
    df_rep["denomination_cor"] = df_rep["denomination"]
    df_rep["denomination_cor"] = df_rep["denomination_cor"].str.lower()
    # Correction des adresses
    df_rep.loc[(df_rep["ville"].isna()) & (df_rep["adresse_ligne3"].isna() == False), "ville"] = \
    df_rep.loc[(df_rep["ville"].isna()) & (df_rep["adresse_ligne3"].isna() == False), "adresse_ligne3"]
    df_rep["commune"] = df_rep["ville"]
    df_rep["commune"] = df_rep["commune"].str.lower()
    df_rep["commune"] = (df_rep["commune"].str.replace("à", "a")
                                            .str.replace("è", "e")
                                            .str.replace("é", "e")
                                            .str.replace("ç", "c")
                                            .str.replace("ù", "u")
                                            .str.replace("ô", "o"))
    df_rep["commune"] = (df_rep["commune"].str.replace("saint", "st"))
    df_rep["commune"] = df_rep["commune"].str.replace(" ", "")
    # mettre date de naissance sous la forme AAAMMJJ
    df_rep["date_naissance_cor"] = df_rep["date_naissance"].str.replace("-","")
    return df_rep

In [42]:
df_rep = clean_rep_inpi(rep)

/tmp/ipykernel_1382/989057859.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
  df_rep["nom_patronymique_cor"] = df_rep["nom_patronymique_cor"].str.replace(re1,"")


In [38]:
# Traitement des particules
df_rep["nom_patronymique_cor"] = (df_rep["nom_patronymique_cor"].str.replace(" DE LA ", "-DE-LA-")
    .str.replace(" DU ", "-DU-").str.replace(" LE ", "-LE-").str.replace(" LA ", "-LA-")
    .str.replace(" EL ", "-EL-")
    .str.replace(" SAINT ", "-SAINT-")
    .str.replace(" VAN ", "-VAN-")
    .str.replace(" DE ", "-DE-")
    .str.replace(" D ", "-D-")
    .str.replace("SAINT ", "SAINT-")
    .str.replace("D ", "D-")
    .str.replace("DE ", "DE-")
    .str.replace("DU ", "DU-")
    .str.replace("LE ", "LE-")
    .str.replace("LA ", "LA-")
    .str.replace("EL ", "EL-")
    .str.replace("DI ", "DI-")
    .str.replace("BEN ", "BEN-")
    .str.replace("VAN ", "VAN-")
    .str.replace(" D'", "-D'"))



In [35]:
df_rep.loc[df_rep["top_particule"] == True, "nom_patronymique2"].head()

57                     DE-LAFORCADE
75     THIERRY-D'ARGENLIEU DE-GYVES
108                        DE-MORAS
114                      DE-LORENZO
115                      DE-LORENZO
Name: nom_patronymique2, dtype: object

In [40]:
df_rep.columns

Index(['code_greffe', 'nom_greffe', 'numero_gestion', 'siren', 'type',
       'nom_patronymique', 'nom_usage', 'pseudonyme', 'prenoms',
       'denomination', 'siren_representant_pm', 'forme_juridique',
       'adresse_ligne1', 'adresse_ligne2', 'adresse_ligne3', 'code_postal',
       'ville', 'code_commune', 'pays', 'date_naissance', 'ville_naissance',
       'pays_naissance', 'nationalite', 'qualite', 'id_representant',
       'date_greffe', 'libelle_evt', 'source', 'year', 'file_path',
       'date_greffe', 'nom_patronymique2', 'top_particule', 'qualite2',
       'denomination2', 'commune', 'date_naissance2'],
      dtype='object')

In [39]:
# traitement des prenoms
df_rep.loc[df_rep["prenoms"] == "", ["nom_patronymique2", "prenoms"]]

,nom_patronymique2,prenoms
24689,SYED-ADIL RASHID,
25196,ISLAM AWLAD,
54431,ABD-EL-SALAM ABD-EL-FATTAH AHMED-RADWAN,
70930,PIERRON,
70931,PIERRON,
...,...,...
11010204,CHAN PEK GNO,
11012712,DE-SEZE AMAURY,
11014150,HO YEN,
11018162,CHEUNG YUI CHI,


In [ ]:
df_rep.loc[df_rep["prenoms"] == "", "nom_patronymique2"].str